# Prototype: Generating Fake Abalone Data (CTGAN)

><div class="alert alert-block alert-info"><b>NOTE: </b>Recommend using an <em>ml.m5.4xlath (16vCPU + 64MB)</em> instance type and, <em>Python 3 (Data Science)</em> kernel to train the <b>CTGAN</b> model.</div> 

In [1]:
# Install CTGAN
#!pip install ctgan

In [2]:
import boto3
import io
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
s3 = boto3.client('s3')

# 'raw' data column names
names = [
    'sex',
    'length',
    'diameter',
    'height',
    'whole_weight',
    'shucked_weight',
    'viscera_weight',
    'shell_weight',
    'rings'
]

# Location of the 'raw' data
bucket = 'data-us-east-2-500842391574'
key = 'input/raw/abalone.csv'
obj = s3.get_object(Bucket=bucket, Key=key)
raw_data = pd.read_csv(io.BytesIO(obj['Body'].read()), encoding='utf8', names=names)
raw_data.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [3]:
# Fit the CTGAN model
from ctgan import CTGANSynthesizer

ctgan = CTGANSynthesizer()
ctgan.fit(raw_data, ['sex'])

In [4]:
# Generate 300 samples from the CTGAN model
samples = ctgan.sample(300)

In [5]:
# Compare the raw data
raw_data.describe()

,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [6]:
# Compare the sample data
samples.describe()

,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,0.505286,0.407042,0.137270,0.920711,0.324504,0.212190,0.262217,10.050000
std,0.138321,0.102988,0.042552,0.540203,0.214242,0.122779,0.151058,3.502627
min,0.080966,0.113664,0.015600,-0.008829,-0.038991,-0.009022,0.016719,4.000000
25%,0.425663,0.354496,0.107900,0.515029,0.161297,0.108346,0.134884,8.000000
50%,0.538431,0.425371,0.144735,0.858251,0.328957,0.220025,0.268548,9.000000
75%,0.612553,0.495828,0.166159,1.282959,0.457227,0.305445,0.358566,12.000000
max,0.739048,0.580600,0.223906,2.490661,1.125791,0.557467,0.856696,22.000000


In [7]:
# Save the samples as fake abalone data
samples.to_csv('fake-abalone.csv', index=False)